# GFS Experiments: RAG Performance Analysis

This notebook conducts systematic experiments with Google Generative File Search.

**Objectives**:
1. Define test query set
2. Measure response latency (P50, P95, P99)
3. Analyze retrieval quality
4. Examine grounding/citations
5. Calculate costs

In [ ]:
import sys
from pathlib import Path
import json
import time
from typing import List, Dict

project_root = Path.cwd().parent
sys.path.append(str(project_root / "src"))

from gfs_client import GFSClient
from utils import load_api_key

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")

print("Imports successful")

## 1. Load Store and Initialize Client

In [ ]:
# Load API key
api_key = load_api_key("GOOGLE_API_KEY", str(project_root / ".env"))
gfs = GFSClient(api_key=api_key)

# Load store metadata
metadata_path = project_root / "models" / "gfs_stores" / "metadata.json"
with open(metadata_path, "r") as f:
    store_metadata = json.load(f)

store_name = store_metadata["store_name"]
print(f"Using store: {store_metadata['display_name']}")
print(f"Store name: {store_name}")

## 2. Define Test Queries

Create a diverse query set covering different patterns:
- Factual lookups
- Analytical questions
- Multi-document reasoning
- Edge cases (out-of-domain)

In [ ]:
# Test query set
test_queries = [
    {
        "id": "q1",
        "query": "What are the main topics covered in the documents?",
        "category": "overview"
    },
    {
        "id": "q2",
        "query": "Summarize the key findings or conclusions.",
        "category": "synthesis"
    },
    {
        "id": "q3",
        "query": "What specific data or statistics are mentioned?",
        "category": "factual"
    },
    {
        "id": "q4",
        "query": "Are there any recommendations or best practices?",
        "category": "actionable"
    },
    {
        "id": "q5",
        "query": "What is the weather forecast for tomorrow?",
        "category": "out_of_domain"
    },
]

print(f"Test queries: {len(test_queries)}")

## 3. Run Experiments

In [ ]:
# Run queries and collect metrics
results = []

for i, test in enumerate(test_queries):
    print(f"\n[{i+1}/{len(test_queries)}] {test['id']}: {test['query'][:50]}...")
    
    try:
        start_time = time.time()
        
        response = gfs.query_with_file_search(
            query=test["query"],
            store_names=[store_name],
            temperature=0.0
        )
        
        latency = time.time() - start_time
        
        # Extract citations
        citations = gfs.extract_citations(response)
        has_citations = citations is not None and citations.get("search_entry_point") is not None
        
        # Count tokens (rough estimate)
        response_text = response.text
        token_count = len(response_text.split()) * 1.3  # Rough approximation
        
        result = {
            "query_id": test["id"],
            "category": test["category"],
            "latency_seconds": latency,
            "response_length": len(response_text),
            "estimated_tokens": int(token_count),
            "has_citations": has_citations,
            "status": "success"
        }
        
        print(f"  ✓ Latency: {latency:.2f}s, Citations: {has_citations}")
        
    except Exception as e:
        result = {
            "query_id": test["id"],
            "category": test["category"],
            "status": "failed",
            "error": str(e)
        }
        print(f"  ✗ Failed: {e}")
    
    results.append(result)
    
    # Rate limiting
    time.sleep(1)

print(f"\nExperiments completed: {len(results)} queries")

## 4. Analyze Performance

In [ ]:
# Calculate latency statistics
successful_results = [r for r in results if r["status"] == "success"]

if successful_results:
    latencies = [r["latency_seconds"] for r in successful_results]
    
    print("Latency Statistics:")
    print(f"  Mean: {np.mean(latencies):.3f}s")
    print(f"  Median (P50): {np.median(latencies):.3f}s")
    print(f"  P95: {np.percentile(latencies, 95):.3f}s")
    print(f"  P99: {np.percentile(latencies, 99):.3f}s")
    print(f"  Min: {np.min(latencies):.3f}s")
    print(f"  Max: {np.max(latencies):.3f}s")
    
    # Citation rate
    citation_rate = sum(r["has_citations"] for r in successful_results) / len(successful_results)
    print(f"\nCitation Rate: {citation_rate*100:.1f}%")
else:
    print("No successful queries to analyze")

In [ ]:
# Visualize latency distribution
if successful_results:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Histogram
    ax1.hist(latencies, bins=10, edgecolor="black", alpha=0.7)
    ax1.axvline(np.median(latencies), color="red", linestyle="--", label="Median")
    ax1.set_xlabel("Latency (seconds)")
    ax1.set_ylabel("Count")
    ax1.set_title("GFS Query Latency Distribution")
    ax1.legend()
    
    # Box plot by category
    categories = [r["category"] for r in successful_results]
    unique_cats = list(set(categories))
    
    cat_latencies = [[r["latency_seconds"] for r in successful_results if r["category"] == cat] 
                     for cat in unique_cats]
    
    ax2.boxplot(cat_latencies, labels=unique_cats)
    ax2.set_ylabel("Latency (seconds)")
    ax2.set_title("Latency by Query Category")
    ax2.tick_params(axis="x", rotation=45)
    
    plt.tight_layout()
    plt.show()

## 5. Cost Analysis

In [ ]:
# Estimate query costs
if successful_results:
    total_tokens = sum(r["estimated_tokens"] for r in successful_results)
    
    # Gemini pricing (example rates)
    # Input: retrieved context counted as input tokens
    # Output: generated response tokens
    # Using approximate rate: $0.075 per 1M input tokens, $0.30 per 1M output tokens
    
    estimated_cost_per_query = (total_tokens / 1_000_000) * 0.30 / len(successful_results)
    
    print("Cost Estimation:")
    print(f"  Total estimated tokens: {total_tokens:,}")
    print(f"  Avg tokens per query: {total_tokens / len(successful_results):.0f}")
    print(f"  Estimated cost per query: ${estimated_cost_per_query:.6f}")
    print(f"  Estimated cost for 1000 queries: ${estimated_cost_per_query * 1000:.2f}")
    print("\nNote: This is a rough estimate. Actual costs depend on retrieved context size.")

## 6. Save Results

In [ ]:
# Save experimental results
results_path = project_root / "models" / "gfs_stores" / "experiment_results.json"

output = {
    "store_name": store_name,
    "model_id": gfs.model_id,
    "queries": test_queries,
    "results": results,
    "summary": {
        "total_queries": len(results),
        "successful": len(successful_results),
        "failed": len(results) - len(successful_results),
        "mean_latency": np.mean(latencies) if successful_results else None,
        "median_latency": np.median(latencies) if successful_results else None,
        "citation_rate": citation_rate if successful_results else None,
    }
}

with open(results_path, "w") as f:
    json.dump(output, f, indent=2)

print(f"Results saved to: {results_path}")

## Summary

**GFS Performance Metrics Collected**:
- Query latency (P50, P95, P99)
- Citation/grounding rate
- Token usage estimates
- Cost projections

**Next Steps**:
- Implement custom RAG baseline in `04_custom_rag_baseline.ipynb`
- Run same query set on custom RAG
- Compare metrics in `05_comparison_analysis.ipynb`